<a href="https://colab.research.google.com/github/bugcaps/workjob/blob/main/%EC%8B%A4%EA%B1%B0%EB%9E%98%EA%B0%80%ED%95%A9%EC%B9%98%EA%B8%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### 실거래가를 년도별 csv화일을 합친다.
* 참조데이터 : 국도교통부 실거래가 자료공개 데이터 cvs 1년단위 download
  * http://rtdown.molit.go.kr/
  * 서울,경기권 거래내역

In [ ]:
# 드라이브  연결
from google.colab import drive
drive.mount('/content/drvie') 

Mounted at /content/drvie


In [ ]:
!ls /content/drvie/MyDrive/apartHome/실거래가

 2019_21_서울경기_40_130.csv
'아파트(매매)__실거래가_2019_서울.csv'
'아파트(매매)__실거래가_2019_경기도.csv'
'아파트(매매)__실거래가_2020_서울.csv'
'아파트(매매)__실거래가_2020_경기도.csv'
'아파트(매매)__실거래가_2021_서울.csv'
'아파트(매매)__실거래가_2021_경기도.csv'
 2021_성동구.csv


In [ ]:

import pandas as pd

df = pd.read_csv('/content/drvie/MyDrive/apartHome/실거래가/아파트(매매)__실거래가_2021_서울.csv',encoding='CP949')
#df.drop("해제사유발생일", axis=1,inplace=True) ## axis=1 열, 0 행을 의미한다. inplace=True 현재 객체에 설정하는 option
#df.set_index("단지명" , inplace=True)  
#display(df)
#display(df.columns)


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42637 entries, 0 to 42636
Data columns (total 15 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   시군구       42637 non-null  object 
 1   번지        42623 non-null  object 
 2   본번        42632 non-null  float64
 3   부번        42632 non-null  float64
 4   단지명       42637 non-null  object 
 5   전용면적(㎡)   42637 non-null  float64
 6   계약년월      42637 non-null  int64  
 7   계약일       42637 non-null  int64  
 8   거래금액(만원)  42637 non-null  object 
 9   층         42637 non-null  int64  
 10  건축년도      42636 non-null  float64
 11  도로명       42637 non-null  object 
 12  해제사유발생일   1168 non-null   float64
 13  거래유형      42637 non-null  object 
 14  중개사소재지    42637 non-null  object 
dtypes: float64(5), int64(3), object(7)
memory usage: 4.9+ MB


## 실거래가 데이터 확인
* 전체 csv 화일에서 필요항목만 추출
  * 시군구','번지','도로명', '단지명','전용면적(㎡)','계약년월','계약일','거래금액(만원)
  * group 항목 추가 : 전용면적을 10단위로 블럭화

In [ ]:
def exportconvertData(df):
  ''' 검색하려는 구를 추출하여 컬럼을 동일하게 정의 '''
  tempdf = df.drop("해제사유발생일", axis=1) ## axis=1 열, 0 행을 의미한다. inplace=True 현재 객체에 설정하는 option
  newdata =tempdf.filter(items=['시군구','번지','도로명', '단지명','전용면적(㎡)','계약년월','계약일','거래금액(만원)'])
  newdata.rename(columns = {'전용면적(㎡)' : '전용면적','거래금액(만원)':'금액'}, inplace = True)

  labels = ["{0} - {1}".format(i, i + 9) for i in range(0, 200, 10)]
  newdata["group"] = pd.cut(newdata['전용면적'], range(0, 205, 10), right=False, labels=labels)  
 ## display(newdata)
  
  ## 데이터를 추출 한다 40~130까지 전용면적
  #exportData =newdata[newdata['전용면적']>40  & ~(newdata['전용면적']<=130)].copy()
  condition = (newdata.전용면적 >40) & (newdata.전용면적<130)
  exportData = newdata[condition].copy()

  ## 데이터 전처리  
  exportData['계약년월'] =pd.to_datetime(exportData['계약년월'], format='%Y%m', errors='ignore')  
  exportData["동"] = exportData.시군구.str.split(" ").str[2]
  ##검색구["주소"]=검색구['시군구'] +' ' + 검색구['도로명']
  exportData['금액']= pd.to_numeric(exportData['금액'].str.replace(',', ''))  ## 숫자로변환
  exportData=exportData.filter(items=['동', '단지명','group','전용면적','계약년월','계약일','금액','시군구','도로명'])
  ##검색구.reset_index()
  return exportData

In [ ]:
## test용
testdata = pd.read_csv('/content/drvie/MyDrive/apartHome/실거래가/아파트(매매)__실거래가_2021_서울.csv',encoding='CP949')
testdata=exportconvertData(testdata)
condition = (testdata.전용면적 >40) & (testdata.전용면적<130)
testdata2 = testdata[condition].copy()
testdata2.info()
groups = testdata2['단지명'].groupby([testdata2['group']])
groups.describe()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 35226 entries, 0 to 42636
Data columns (total 9 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   동       35226 non-null  object        
 1   단지명     35226 non-null  object        
 2   group   35226 non-null  category      
 3   전용면적    35226 non-null  float64       
 4   계약년월    35226 non-null  datetime64[ns]
 5   계약일     35226 non-null  int64         
 6   금액      35226 non-null  int64         
 7   시군구     35226 non-null  object        
 8   도로명     35226 non-null  object        
dtypes: category(1), datetime64[ns](1), float64(1), int64(2), object(4)
memory usage: 2.5+ MB


,count,unique,top,freq
group,,,,
0 - 9,0,0,NaN,NaN
10 - 19,0,0,NaN,NaN
20 - 29,0,0,NaN,NaN
30 - 39,0,0,NaN,NaN
40 - 49,3339,452,계림웨스트벨리,143
50 - 59,11174,1672,두산,197
60 - 69,1506,518,한신,36
70 - 79,2156,734,엘에이치강남아이파크,168
80 - 89,12929,2469,현대,180


In [ ]:


years = ['2019','2020','2021']
areas =['서울','경기도']

alldf = pd.DataFrame()

for yyyy in years: 
  for a in areas:
    path ='/content/drvie/MyDrive/apartHome/실거래가/아파트(매매)__실거래가_'+ yyyy+'_' + a +'.csv'
   
    temp = pd.read_csv(path,encoding='CP949') 
    temp=exportconvertData(temp)
    alldf = pd.concat([alldf, temp], ignore_index = True, axis = 0)

   
    ##df = pd.read_csv('/content/drvie/MyDrive/apartHome/실거래가/아파트(매매)__실거래가_2021_서울.csv',encoding='CP949')
alldf.info()


/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (2) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (2) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 672828 entries, 0 to 672827
Data columns (total 9 columns):
 #   Column  Non-Null Count   Dtype         
---  ------  --------------   -----         
 0   동       672828 non-null  object        
 1   단지명     672828 non-null  object        
 2   group   672828 non-null  category      
 3   전용면적    672828 non-null  float64       
 4   계약년월    672828 non-null  datetime64[ns]
 5   계약일     672828 non-null  int64         
 6   금액      672828 non-null  int64         
 7   시군구     672828 non-null  object        
 8   도로명     672828 non-null  object        
dtypes: category(1), datetime64[ns](1), float64(1), int64(2), object(4)
memory usage: 41.7+ MB


### 화일생성완료

In [ ]:
#csv 파일 생성
alldf.to_csv("/content/drvie/MyDrive/apartHome/실거래가/2019_21_서울경기_40_130.csv",index=False)


In [ ]:
!ls /content/drvie/MyDrive/apartHome/실거래가

 2019_21_서울경기_40_130.csv
'아파트(매매)__실거래가_2019_서울.csv'
'아파트(매매)__실거래가_2019_경기도.csv'
'아파트(매매)__실거래가_2020_서울.csv'
'아파트(매매)__실거래가_2020_경기도.csv'
'아파트(매매)__실거래가_2021_서울.csv'
'아파트(매매)__실거래가_2021_경기도.csv'
 2021_성동구.csv


## 위도경도 좌표설정을 위한 건물물 단위 Group by

In [ ]:
apt = pd.read_csv('/content/drvie/MyDrive/apartHome/실거래가/2019_21_서울경기_40_130.csv')
apt["주소"]=apt['시군구'] +' ' + apt['도로명']
apt

,동,단지명,group,전용면적,계약년월,계약일,금액,시군구,도로명,주소
0,개포동,개포2차현대아파트(220),70 - 79,77.750,2019-06-01,18,134500,서울특별시 강남구 개포동,언주로 103,서울특별시 강남구 개포동 언주로 103
1,개포동,개포2차현대아파트(220),70 - 79,77.750,2019-11-01,18,160000,서울특별시 강남구 개포동,언주로 103,서울특별시 강남구 개포동 언주로 103
2,개포동,개포6차우성아파트1동~8동,60 - 69,67.280,2019-05-01,4,124000,서울특별시 강남구 개포동,언주로 3,서울특별시 강남구 개포동 언주로 3
3,개포동,개포6차우성아파트1동~8동,70 - 79,79.970,2019-06-01,15,141000,서울특별시 강남구 개포동,언주로 3,서울특별시 강남구 개포동 언주로 3
4,개포동,개포6차우성아파트1동~8동,70 - 79,79.970,2019-08-01,28,155000,서울특별시 강남구 개포동,언주로 3,서울특별시 강남구 개포동 언주로 3
...,...,...,...,...,...,...,...,...,...,...
672823,향남읍,화성향남시범복사꽃마을한국아델리,80 - 89,84.956,2021-07-01,6,34000,경기도 화성시 향남읍 행정리,행정동로 88,경기도 화성시 향남읍 행정리 행정동로 88
672824,향남읍,화성향남시범복사꽃마을한국아델리,80 - 89,84.971,2021-07-01,8,39000,경기도 화성시 향남읍 행정리,행정동로 88,경기도 화성시 향남읍 행정리 행정동로 88
672825,향남읍,화성향남시범복사꽃마을한국아델리,80 - 89,84.956,2021-07-01,22,38800,경기도 화성시 향남읍 행정리,행정동로 88,경기도 화성시 향남읍 행정리 행정동로 88
672826,향남읍,화성향남시범복사꽃마을한국아델리,80 - 89,84.994,2021-07-01,31,34950,경기도 화성시 향남읍 행정리,행정동로 88,경기도 화성시 향남읍 행정리 행정동로 88


### 좌표정보를 조회하기위한 주소정보 중복제거:::
* Pandas에서는 두 개의 함수를 제공 - unique()와 drop_duplicates() 
두 함수의 차이는?
* 데이터프레임['컬럼'].unique() 
  - 고유 값들을 array형으로 반환
* 데이터프레임['컬럼'].drop_duplicate()
  - 고유 값들을 Series형으로 반환

In [ ]:

#drop_duplicates 을 제거한 data를 추출 한다. 동 명을 추출한다.
#uniques = apt.drop_duplicates(subset=['주소','단지명']).reset_index()
uniques = apt['주소'].drop_duplicates().reset_index()
del uniques['index']
uniques.reset_index(drop=True, inplace=True)
uniques.to_csv("/content/drvie/MyDrive/apartHome/실거래가/서울경기아파트주소.csv",index=False)